In [ ]:
%%html
<style>
  table {margin-left: 0!important; width: 75%;}
</style>
<style>
  td {font-size: 14px;}
</style>

# GRO620 - Activité procédurale 1

Dans cette activité, nous allons principalement travailler sur les éléments nécessaires pour capter une image numériquement, les transformations entre repères 2D et 3D, et l'encodage numérique de la couleur.

In [ ]:
# Préambule

import numpy as np
import cv2

import matplotlib.pyplot as plt
%matplotlib inline

## Si vous utilisez Google Colab, vous devez d'abord monter votre Google Drive
## où se trouve vos données. 
## Commentez les trois lignes suivantes en ajustant le chemin vers votre propre
## dossier :

# from google.colab import drive
# drive.mount('/content/gdrive')
# %cd /content/gdrive/MyDrive/gro620-e21

## Pour retrouver le chemin depuis Jupyter, vous pouvez utiliser ceci :
# !ls /content/gdrive/MyDrive


## Acquisition

### Q1.1

À partir de la figure 2.23 du livre de référence, décrivez en une phrase le rôle de chacune des étapes de la chaîne d'acquisition d'images numériques.

| Acronym | Definition                                     |
| :------ | :--------------------------------------------- |
| A/D     | Analog-to-Digital                              |
| CCD     | Charge-Coupled Device                          |
| CMOS    | Complementary Metal–Oxide–Semiconductor        |
| DSP     | Digital Signal Processing                      |
| ISO     | International Organization for Standardization |

#### Camera Body

- Optics: Focaliser la lumière sur le capteur.
- Aperture: Contrôler la quantité de lumière entrante.
- Shutter: Contrôler la durée d'exposition de la lumière sur le capteur.

#### Sensor Chip

- Sensor (CCD/CMOS): Capter la lumière.
- Gain (ISO): Amplifier le signal analogique (sensibilité de détection).
- A/D Converter: Convertir le signal analogique en signal numérique.

#### DSP

- Demosaic: Reconstruction de l'image couleur avec les échantillons provenant du capteur. On peut aussi parler d'interpolation.
- Sharpen: Rehausser les différences en augmentant les hautes fréquences (filtre).
- White Balance: Modifier l'éclairage ou la température de la photo.
- Gamma / Curve Correction: Modification de la luminance avec une loi de puissance.
- Compression: Encoder l'information de l'image de façon à réduire le nombre de bits requis.

### Q1.2

Quelle est la différence entre paramètres intrinsèques et extrinsèques d'une caméra ? Décrivez chaque type en une phrase.

- Intrinsèque: propre à la caméra, interne (ex.: la taille du capteur).
- Extrinsèque: se rapporte à la caméra par rapport à son environnement, externe (ex.: la position de la caméra).

### Q1.3

Soit la configuration intrinsèque d'une caméra représentée par la matrice $K$ :

$$
K = \begin{bmatrix} 
 620 &   0 & 1024 \\ 
   0 & 620 &  512 \\ 
   0 &   0 &    1 
\end{bmatrix}
$$

Le capteur de cette caméra a une taille de 30 mm x 15 mm.

Pouvez-vous estimer la distance focale en mm de la lentille de cette caméra à partir de la matrice $K$ ?

In [ ]:
# Focal length:  620px
# Half-width:   1024px or 30mm
# Half-height:   512px 04 15mm

K = np.array([[620.,   0., 1024.],
              [  0., 620.,  512.],
              [  0.,   0.,    1.]
])

f = 30*K[0,0]/(2*K[0,2])
print(f"Distance focale: {f:.3f} mm")

### Q1.4

Dans le cadre de cet APP, nous considérons les caméras comme étant idéales, c'est-à-dire qu'on peut obtenir leurs caractéristiques intrinsèques et extrinsèques à partir de quelques paramètres seulement.

**a)** Qu'est-ce qui rend les vraies caméras non-idéales ? Nommez des facteurs autant pour les caractéristiques intrinsèques que extrinsèques.

- *Skew*, ou un mauvais alignement entre l'axe optique et la normale du capteur
- Distorsion radiale (ex.: *pincushion* ou *barrel*)
- Tolérances durant le processus de fabrication
- Position de la caméra par rapport au repère global

**b)** Que doit on faire pour obtenir les caractéristiques d'une caméra non-idéale ?

Une calibration

### Q1.5

**a)** Pourquoi deux appareils de capture peuvent produire des valeurs RGB différentes d'une même couleur ? 

Suffit qu'un seul aspect varie dans la chaîne d'acquisition en amont pour produire des couleurs numériques différentes.

**b)** Que peut-on faire pour comparer numériquement des couleurs provenant de deux capteurs différents ?

*Mapping* des couleurs vers un espace commun XYZ (chaque couleur est sur un spectre de 0 à 1). On peut aussi utiliser un espace L\*a\*b\*, ou la distance euclidienne varie selon la perception de l'oeil humain.

## Repères et coordonnées

### Q2.1

Supposons ces 2 repères :

![](images_doc/proc1-q2_1-frames.png)

**a)** Trouvez la matrice homogène permettant de transformer un point du repère $\{1\}$ au repère $\{0\}$.

In [ ]:
T_10 = np.identity(4) # Génère une matrice identité 4x4

T_10 = np.array([[0,1,0,240],[1,0,0,80],[0,0,-1,120],[0,0,0,1]])

print("T_10:\n", T_10)

**b)** Trouvez maintenant la transformation inverse.

In [ ]:
T_01 = np.identity(4)
T_01 = np.linalg.inv(T_10)
print("T_01:\n", T_01)

**c)** Soit le point $p_0 = [8, 5, 1]^T$, un point dans le repère $\{0\}$. Trouvez $p_1$, ses coordonnées dans le repère $\{1\}$.

In [ ]:
p_0 = [8, 5, 1]
p_1 = [0, 0, 0]

p_0_h = [8,5,1,1]
p_1_h = T_01 @ p_0_h

p_1 = [k for k in p_1_h[0:-1]]
print(p_1)

### Q2.2

Supposons maintenant que le repère $\{1\}$ représente une caméra avec les caractéristiques intrinsèques $K$ de la question Q1.3.

**a)** Trouvez la matrice de projection P complète permettant de projeter un point $p$ décrit dans le repère $\{0\}$.

In [ ]:
# print(K) # Si vous n'avez pas réutilisé la variable K, elle aura toujours la même valeur qu'à la question Q1.3.

R = np.array([T_01[0][0:-1],T_01[1][0:-1],T_01[2][0:-1]])
t = np.array(T_01.T[-1][0:-1])

Kt = np.zeros([4,4])
Kt[0:-1,0:-1] = K
Kt[-1,-1] = 1

Rt = np.zeros([4,4])
Rt[0:-1,0:-1] = R
Rt[0:-1,-1] = t
Rt[-1,-1] = 1

Pt = Kt @ Rt
print(Pt)

**b)** Soit le point $p_0 = [0.250, 0.010, 0.000]$. Trouvez le point $x_s$, les coordonnées du point $p_0$ perçu par la caméra.

On veut trouver ses coordonnées dans l'image. On peut utiliser l'équation (2.65) du livre. Pour cela, il faut d'abord bâtir Pt ($\tilde{P}$), qui dépend également de K. Rappel des équations :


$$P  = K \left[R \middle| T\right]$$

$$\tilde{P} = \tilde{K} E$$

$$x_s \sim \tilde{P} \bar{p_w}$$

$$x_s = \left(X_s, Y_s, 1, d\right)$$

$$d = \frac{1}{z}$$

Dans notre cas, $E$ correspond à $T_{01}$, car on souhaite passer du repère global à celui de la caméra, $p_w$ est donc $p_0$ et $x_s$ continendra les coordonnées normalisées dans l'image.

$d$ correspond à la disparité, ou $1/z$, où $z$ et la distance du point dans le repère de la caméra (donc en Z, où Z augmente avec la distance).

Nous pouvons maintenant bâtir les matrices :

In [ ]:
p_0 = np.array([0.250, 0.010, 0.000])

p_0_h = np.ones(4)
p_0_h[0:-1] = p_0

xs = Pt.dot(p_0_h)
xs /= xs[2]
print(xs[0:-1])
print(f"Camera / image distance: {1/xs[-1]:.3f}")

Si on arrondit, on obtient (611, -727), ce qui donne un point en dehors du capteur. Le point $p_0$ est donc hors du champ de vision. En effet, les coordonnées sont référencées au coint supérieur gauche de l'image.

Si on regarde les repères, notre point est situé sur l'axe négatif en Y, ce qui veut dire qu'il est trop "en haut" de la caméra. Sur le repère ${0}$, ceci correspond à un point probablement trop près du référentiel sur l'axe des X. Nous allons donc rapprocher le point sur l'axe des X pour tenter de trouver une coordonnée image plus intéressante :

In [ ]:
p1w = np.array([300,0.010,0.001,1])
x1_s_p = Pt @ p1w
x1_s = x1_s_p / x1_s_p[2]
print(x1_s[0:-1])

## Reprojection 2D à 3D

### Q3.1

Supposons que le plan XY du repère $\{0\}$ est un convoyeur. Quelle serait sa largeur maximale (mesurée sur l'axe Y) si on souhaite que la caméra la capte au complet dans son image ? 

In [ ]:
l_conv = 0



### Q3.2

Soit le point $x_s = [120, 200]$, un point dans l'image perçu par la caméra décrite plus haut. On suppose que le point perçu se trouve sur le plan XY du repère $\{0\}$. Trouvez les coordonnées du point $p_0$ qui correspond à ce même point dans le repère $\{0\}$.

In [ ]:
x_s = np.array([120,200, 1, 1/120])

Pt_inv = np.linalg.inv(Pt)
print("Pt_inv:\n:", Pt_inv)

(Attention: ici, on ne peut pas simplement transposer comme si c'était un repère)

En inversant, on repasse d'un système en pixels à un en mm. Or, on ne peut pas tout de suite multiplier $\tilde{P}^{-1}$ et un vecteur de coordonnées en pixels. On se rappelle que cette matrice ne manipule que les coordonnées normalisées. Il faut donc remultiplier par $z$, ou diviser par $d$ (ce qui correspond à diviser par $1/z$). On peut s'en convaincre avec le point précédent :

In [ ]:
x_rp = x_s / x_s[3] # rp pour "reprojection"
print(x_rp) # Devrait corresponde à x_s_p
p_0_rp = np.matmul(Pt_inv, x_rp)
print(p_0_rp)

On obtient donc un point franchement vers Y- (ce qui correspond à la gauche de l'image) et à mi-chemin entre les deux repères en X, ce qui a du sens compte tenu des coordonnées de l'image.